 starting with a graph with two nodes connected by one edge.


In [27]:
#!pip install langgraph

Nodes act like functions that can be called as needed. considering Node 1 is our starting point and Node 2 is END point.

In [28]:
def function_1(input_1):
    return input_1 + " Hello "

def function_2(input_2):
    return input_2 + "how are you"

In [29]:
from langgraph.graph import Graph

flow = Graph()

flow.add_node("node_1", function_1)
flow.add_node("node_2", function_2)

flow.add_edge('node_1', 'node_2')

flow.set_entry_point("node_1")
flow.set_finish_point("node_2")

app = flow.compile()

In [30]:
app.invoke("vidya:")

'vidya: Hello how are you'

In [31]:
input = 'vidya:'
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'node_1':
---
vidya: Hello 

---

Output from node 'node_2':
---
vidya: Hello how are you

---



# Adding LLM Call

making the first node as an AGent that can call Open AI models.

In [32]:
#!pip install langchain langchain_openai

In [33]:
!pip install python-dotenv

In [34]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key= os.environ.get("OPENAI_API_KEY")

In [35]:

from langchain.chat_models import AzureChatOpenAI

llm_model1 = AzureChatOpenAI(
    deployment_name=os.environ.get("deployment_name"),
    model_name=os.environ.get("model_name"),
    temperature=os.environ.get("temperature"),
    openai_api_base= os.environ.get("openai_api_base"),
    openai_api_version=os.environ.get("openai_api_version"),
    openai_api_key=os.environ.get("openai_api_key"),
    openai_api_type=os.environ.get("openai_api_type"),
)

#giving user message
llm_model1.invoke('Hey there')

/home/karuturividyasagar/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://magicplatform-dev-canadaeast.openai.azure.com to https://magicplatform-dev-canadaeast.openai.azure.com/openai.
  warnings.warn(
/home/karuturividyasagar/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/home/karuturividyasagar/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f336653-b567-45f2-9c78-cc01626f481a-0')

And if you just want to see the AI response, you can do the following:

In [36]:
llm_model1.invoke('Hey there').content

'Hello! How can I assist you today?'

In [37]:
def function_1(input_1):
    response = llm_model1.invoke(input_1)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [38]:
#Langchain graph
flow = Graph()

#calling node 1 as agent
flow.add_node("agent", function_1)
flow.add_node("node_2", function_2)

flow.add_edge('agent', 'node_2')

flow.set_entry_point("agent")
flow.set_finish_point("node_2")

app = flow.compile()

In [39]:
app.invoke("Hey there")

'Agent Says: Hello! How can I assist you today?'

In [40]:
input = 'Hey there'
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
Hello! How can I assist you today?

---

Output from node 'node_2':
---
Agent Says: Hello! How can I assist you today?

---



# 1st functional Agent App - City Temperature

In [41]:
def function1(input1):
    query="your task is to providde onlly the city name based on the user query. \
        don't give anything more just give me the city name mentioned. This is the users query: " + input1
    response=llm_model1.invoke(query)
    return response.content

# input1=("what is hyderabad weather condition right now")
# print(function1(input1))

def function2(input2):
    return "Agent says: " + input2

In [42]:
flow=Graph()
flow.add_node("agent", function1)
flow.add_node("node2", function2)
flow.add_edge("agent", "node2")
flow.set_entry_point("agent")
flow.set_finish_point("node2")
app=flow.compile()

In [43]:
app

CompiledGraph(nodes={'agent': PregelNode(config={'tags': []}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<agent>(writes=[ChannelWriteEntry(channel='agent', value=None, skip_none=False)])]), 'node2': PregelNode(config={'tags': []}, channels=['agent'], triggers=['agent'], writers=[ChannelWrite<node2>(writes=[ChannelWriteEntry(channel='node2', value=None, skip_none=False)]), ChannelWrite<__end__>(writes=[ChannelWriteEntry(channel='__end__', value=None, skip_none=False)])])}, channels={'__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fbd69c0fa60>, '__end__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fbd69c0f790>, 'agent': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fbd69c0ec80>, 'node2': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fbd69c3c100>}, auto_validate=False, output_channels='__end__', stream_channels=['agent', 'node2'], input_channels='__start__', graph=<langgraph.gra

In [44]:
app.invoke("What's the temperature in chennai")

'Agent says: Chennai'


2: Adding a weather API call


In [45]:
#!pip install pyowm

In [46]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
load_dotenv()
os.environ["OPENWEATHERMAP_API_KEY"] = os.environ.get("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper()

In [47]:
weather_data = weather.run("chennai")
print(weather_data)

In chennai, the current weather is as follows:
Detailed status: few clouds
Wind speed: 6.26 m/s, direction: 180°
Humidity: 57%
Temperature: 
  - Current: 34.7°C
  - High: 35.0°C
  - Low: 33.99°C
  - Feels like: 41.7°C
Rain: {}
Heat index: None
Cloud cover: 20%


integrating this into function 2 and call the function two as a "tool" or "weather_agent" instead of "node_2" in flow.

In [48]:
def function1(input1):
    query="your task is to providde onlly the city name based on the user query. \
        don't give anything more just give me the city name mentioned. This is the users query: " + input1
    response=llm_model1.invoke(query)
    return response.content

def function2(input2):
    weather_data = weather.run(input2)
    return weather_data

In [49]:
from langgraph.graph import Graph

flow = Graph()

#calling node 1 as agent
flow.add_node("agent", function1)
flow.add_node("tool", function2)

flow.add_edge("agent","tool")

flow.set_entry_point("agent")
flow.set_finish_point("tool")

app = flow.compile()

In [50]:
app.invoke("What's the current temperature in Chennai, India")

'In Chennai, the current weather is as follows:\nDetailed status: few clouds\nWind speed: 6.26 m/s, direction: 180°\nHumidity: 57%\nTemperature: \n  - Current: 34.7°C\n  - High: 35.0°C\n  - Low: 33.99°C\n  - Feels like: 41.7°C\nRain: {}\nHeat index: None\nCloud cover: 20%'

In [51]:
input = "What's the temperature in chennai"
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
Chennai

---

Output from node 'tool':
---
In Chennai, the current weather is as follows:
Detailed status: few clouds
Wind speed: 6.26 m/s, direction: 180°
Humidity: 57%
Temperature: 
  - Current: 34.7°C
  - High: 35.0°C
  - Low: 33.99°C
  - Feels like: 41.7°C
Rain: {}
Heat index: None
Cloud cover: 20%

---



 3 Adding another LLM Call to filter results

only the temperature needed , taking another LLM call to filter data

In [52]:
def function_3(input_3):
    complete_query = "Your task is to provide info concisely based on the user query. Following is the user query: " + "user input"
    response = llm_model1.invoke(complete_query)
    return response.content

But the issue is the user input is not available from node 2.so should pass the user input all along from first node to the last?

using dictionary and passing it between nodes (we could also use just a list, but dict is easier)

In [53]:
# an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes.
AgentState["messages"] = []

In [54]:
AgentState

{'messages': []}

goal is to have this state filled as: {'messages': [HumanMessage, AIMessage, ...]}

Also need to modify our functions to pass info along the new AgentState

In [55]:
def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = "Your task is to provide only the city name based on the user query. \
                    Nothing more, just the city name mentioned. Following is the user query: " + user_input
    response = llm_model1.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide info concisely based on the user query and the available information from the internet. \
                        Following is the user query: " + user_input + " Available information: " + available_info
    response = llm_model1.invoke(agent2_query)
    return response.content


In [56]:
from langgraph.graph import Graph

flow = Graph()


flow.add_node("agent", function_1)
flow.add_node("tool", function_2)
flow.add_node("responder", function_3)

flow.add_edge('agent', 'tool')
flow.add_edge('tool', 'responder')

flow.set_entry_point("agent")
flow.set_finish_point("responder")

app = flow.compile()

In [57]:
inputs = {"messages": ["what is the temperature in hyderabad"]}
app.invoke(inputs)

'The current temperature in Hyderabad is 36.23°C with a few clouds. The high temperature for today is 36.73°C and the low temperature is 36.23°C. The humidity is 32% and the wind speed is 4.12 m/s with a direction of 250°. There is no rain and the heat index is none. The cloud cover is 20%.'

In [58]:
input = {"messages": ["what is the temperature in hyderabad"]}
for output in app.stream(input):
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': ['what is the temperature in hyderabad', 'Hyderabad']}

---

Output from node 'tool':
---
{'messages': ['what is the temperature in hyderabad', 'Hyderabad', 'In Hyderabad, the current weather is as follows:\nDetailed status: few clouds\nWind speed: 4.12 m/s, direction: 250°\nHumidity: 32%\nTemperature: \n  - Current: 36.23°C\n  - High: 36.73°C\n  - Low: 36.23°C\n  - Feels like: 37.05°C\nRain: {}\nHeat index: None\nCloud cover: 20%']}

---

Output from node 'responder':
---
The current temperature in Hyderabad is 36.23°C with a few clouds. The wind speed is 4.12 m/s coming from the direction of 250°. The humidity is 32%. The high temperature for today is 36.73°C and the low temperature is 36.23°C. The heat index is not available. There is no rain at the moment and the cloud cover is 20%.

---




app is not capable of answering simple questions like "how are you?"

In [59]:
# inputs = {"messages": ["how are you?"]}
# app.invoke(inputs)

This is because we always want to parse a city and then find the weather.

We can make our agent smarter by saying only use the tool when needed, if not just respond back to the user.

The way we can do this LangGraph is:
1. binding a tool to the agent
2. adding a conditional edge to the agent with the option to either call the tool or not
3. defining the criteria for the conditional edge as when to call the tool. We will define a function for this.

In [60]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

Binding tool with agent (LLM Model) is made easy in langchain

In [61]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [OpenWeatherMapQueryRun()]

#model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = llm_model1.bind_functions(functions)

Our modified function_1 now becomes as below. The reason is, we are passing the human message as state and appending response to the state. Also, our agent now has a tool bound to it, that it can use.

In [62]:
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

For function 2, we want it to setup a tool and call it. It's made easy to invoke a tool in LangChain by using ToolInvocation and executing it with ToolExecuter. Then we respond back as a FunctionMessage so that our agent (node 1) knows that the tool was used and a response from tool is available.

In [63]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )

    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

Finally, we define a function for the conditional edge, to help us figure out which direction to go (tool or user response)

We can benefit from the agent (LLM) response in LangChain, which has additional_kwargs to make a function_call with the name of the tool.

So our logic is, if function_call available in the additional_kwargs, then call tool if not then end the discussion and respond back to the user

In [64]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]

    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

Now with all of the changes above, our LangGraph app is modified as below:

In [65]:
# from langgraph.graph import Graph, END

# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next.
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition,
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()

We also pass the first message using HumanMessage component available in langchain, makes it easy to differentiate from AIMessage, and FunctionMessage

In [66]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the temperature in las vegas'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "__arg1": "Las Vegas"\n}', 'name': 'open_weather_map'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 82, 'total_tokens': 100}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4a739a35-b856-4986-9501-db4a2f2db105-0'),
  FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.81 m/s, direction: 230°\nHumidity: 19%\nTemperature: \n  - Current: 24.4°C\n  - High: 25.45°C\n  - Low: 23.24°C\n  - Feels like: 23.39°C\nRain: {}\nHeat index: None\nCloud cover: 75%', name='open_weather_map'),
  AIMessage(content='The current temperature in Las Vegas is 24.4°C.', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 207, 'total_tokens': 221}, 'model_name': 

In [67]:
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "__arg1": "Las Vegas"\n}', 'name': 'open_weather_map'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 82, 'total_tokens': 100}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c2597e7e-0143-4ba0-88e5-3350a59efdf8-0')]}

---

Output from node 'tool':
---
{'messages': [FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.81 m/s, direction: 230°\nHumidity: 19%\nTemperature: \n  - Current: 24.4°C\n  - High: 25.45°C\n  - Low: 23.24°C\n  - Feels like: 23.39°C\nRain: {}\nHeat index: None\nCloud cover: 75%', name='open_weather_map')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='The current temperature in Las Vegas is 24.4°C.', response_metadata={'token_usage': {'completion_tokens':

Hopefully, that gives you a good understanding of how we built a LangGraph app and why we used different LC components.